# Ecological Inference through Tsallis Regularized Optimal Transport (TROT)
This notebook presents the pipeline used in (cite our paper) to perform ecological inference on the Florida dataset.

You will first want to download the dataset from (url to the dataset)

In [24]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
from matplotlib.pylab import savefig

import sys
sys.path.append('..')
sys.path.append('../Trot')
sys.path.append('../Data')

from Trot import Distances as dist
from Trot.Evaluation import KL
from Trot.Florida_inference import CV_Local_Inference, Local_Inference

# Data Loading and Processing

In [25]:
FlData = pd.read_csv('../Fl_Data.csv', usecols = ['District', 'County','Voters_Age', 'Voters_Gender', 'PID', 'vote08', 
                    'SR.WHI', 'SR.BLA', 'SR.HIS', 'SR.ASI', 'SR.NAT', 'SR.OTH']) 

FlData = FlData.dropna()

Change gender values to numerical values

In [26]:
FlData['Voters_Gender'] = FlData['Voters_Gender'].map({'M': 1, 'F': 0})

Renormalize the age so that it takes values between 0 and 1

In [27]:
FlData['Voters_Age'] = ((FlData['Voters_Age'] -
                         FlData['Voters_Age'].min()) /
                        (FlData['Voters_Age'].max() -
                         FlData['Voters_Age'].min()))


One-hot party subscriptions (PID)

In [28]:
#Get one hot encoding of column PID
one_hot = pd.get_dummies(FlData['PID'])
# Drop column PID as it is now encoded
FlData = FlData.drop('PID', axis=1)
# Join the encoded df
FlData = FlData.join(one_hot)
# Rename the new columns
FlData.rename(columns={0: 'Other', 1: 'Democrat', 2: 'Republican'},
              inplace=True)

In [29]:
FlData.describe()

,District,County,Voters_Age,Voters_Gender,vote08,SR.WHI,SR.BLA,SR.HIS,SR.ASI,SR.NAT,SR.OTH,Other,Democrat,Republican
count,18205.000000,18205,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000,18205.000000
mean,5.429827,12,0.361123,0.473002,0.820159,0.768745,0.103927,0.049767,0.026476,0.003516,0.049602,0.199396,0.543257,0.257347
std,1.177072,0,0.240638,0.499284,0.384065,0.421647,0.305175,0.217468,0.160551,0.059189,0.217127,0.399557,0.498139,0.437184
min,3.000000,12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,12,0.125000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,12,0.350000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,6.000000,12,0.550000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,6.000000,12,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Compute Marginals and Joint Distributions

Create a county dictionnary

In [30]:
Voters_By_County = {}
all_counties = FlData.County.unique()
for county in all_counties:
    Voters_By_County[county] = FlData[FlData['County'] == county]

Compute the ground truth joint distribution

In [31]:
J = {}
for county in all_counties:
    J[county] = np.zeros((6, 3))

    J[county][0,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]
    J[county][0,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]
    J[county][0,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]

    J[county][1,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]
    J[county][1,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]
    J[county][1,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]

    J[county][2,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]
    J[county][2,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]
    J[county][2,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]

    J[county][3,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]
    J[county][3,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]
    J[county][3,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]

    J[county][4,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) &(Voters_By_County[county]['SR.NAT']==1)].shape[0]
    J[county][4,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.NAT']==1)].shape[0]
    J[county][4,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.NAT']==1)].shape[0]

    J[county][5,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]
    J[county][5,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]
    J[county][5,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]

    J[county] /= J[county].sum()

In [32]:
print(J[12])

[[ 0.14225414  0.39540621  0.22952527]
 [ 0.01178599  0.08853196  0.00339875]
 [ 0.01584256  0.02318825  0.0106348 ]
 [ 0.01074444  0.01057998  0.00509813]
 [ 0.00076746  0.00197347  0.00076746]
 [ 0.01792567  0.02351716  0.00805833]]


Compute the party marginals

In [33]:
Party_Marginals = {}
parties = ['Other', 'Democrat', 'Republican']
for county in all_counties:
    Party_Marginals[county] = pd.Series([J[county][:, i].sum()
                                        for i in np.arange(3)])
    Party_Marginals[county].index = parties

Compute the ethnicity marginals

In [34]:
Ethnicity_Marginals = {}
ethnies = ['SR.WHI', 'SR.BLA', 'SR.HIS', 'SR.ASI', 'SR.NAT', 'SR.OTH']
for county in all_counties:
    Ethnicity_Marginals[county] = pd.Series([J[county][i, :].sum()
                                             for i in np.arange(6)])
    Ethnicity_Marginals[county].index = ethnies

# Compute the cost matrix
Using only age, gender, and 2008 vote or abstention

In [35]:
features = ['Voters_Age', 'Voters_Gender', 'vote08']
e_len, p_len = len(ethnies), len(parties)
M = np.zeros((e_len, p_len))
for i, e in enumerate(ethnies):
    data_e = FlData[FlData[e] == 1.0]
    average_by_e = data_e[features].mean(axis=0)
    for j, p in enumerate(parties):
        data_p = FlData[FlData[p] == 1.0]
        average_by_p = data_p[features].mean(axis=0)

        M[i, j] = np.array(dist.dist_2(average_by_e, average_by_p))

# Start the inference

Use a specific county or district to select the best parameters

In [36]:
CV_counties = FlData[FlData['District'] == 3].County.unique()

Find the best parameters

In [37]:
q = np.arange(0.5, 2.1, 0.1)
l = [0.01, 0.1, 1., 10., 100.] 

best_score, best_q, best_l = CV_Local_Inference(Voters_By_County, M, J, Ethnicity_Marginals, Party_Marginals,
                   CV_counties,q,l)

q: 0.50, lambda: 0.0100, KL: 0.07895, STD: 0
q: 0.50, lambda: 0.1000, KL: 0.07859, STD: 0
q: 0.50, lambda: 1.0000, KL: 0.07556, STD: 0
q: 0.50, lambda: 10.0000, KL: 0.1046, STD: 0
q: 0.50, lambda: 100.0000, KL: 0.3701, STD: 0
q: 0.60, lambda: 0.0100, KL: 0.07369, STD: 0
q: 0.60, lambda: 0.1000, KL: 0.07325, STD: 0
q: 0.60, lambda: 1.0000, KL: 0.06959, STD: 0
q: 0.60, lambda: 10.0000, KL: 0.1099, STD: 0
q: 0.60, lambda: 100.0000, KL: 0.3799, STD: 0
q: 0.70, lambda: 0.0100, KL: 0.06793, STD: 0
q: 0.70, lambda: 0.1000, KL: 0.06737, STD: 0
q: 0.70, lambda: 1.0000, KL: 0.0628, STD: 0
q: 0.70, lambda: 10.0000, KL: 0.1208, STD: 0
q: 0.70, lambda: 100.0000, KL: 0.3884, STD: 0
q: 0.80, lambda: 0.0100, KL: 0.06197, STD: 0
q: 0.80, lambda: 0.1000, KL: 0.06123, STD: 0
q: 0.80, lambda: 1.0000, KL: 0.05535, STD: 0
q: 0.80, lambda: 10.0000, KL: 0.1395, STD: 0
q: 0.80, lambda: 100.0000, KL: 0.3934, STD: 0
q: 0.90, lambda: 0.0100, KL: 0.05649, STD: 0
q: 0.90, lambda: 0.1000, KL: 0.05549, STD: 0
q: 0.90

Use selected parameters on the rest of the dataset

In [38]:
J_inferred = Local_Inference(Voters_By_County, M, J, Ethnicity_Marginals, Party_Marginals, all_counties, best_q, best_l)
kl, std = KL(J, J_inferred, all_counties, save_to_file=False, compute_abs_err=True)

Absolute error 0.0075675635082  +  0.0


# Plot the results

In [39]:
diag = np.linspace(-0.1, 1.0, 100)

# pickle results
f = open('../Data/joints_gallup.pkl', 'rb')
J_true, J = pickle.load(f)

f = open('../Data/baseline.pkl', 'rb')
J_baseline = pickle.load(f)

j_true, j, j_baseline = [], [], []
for c in all_counties:
    j_true.append(np.array(J_true[c]).flatten())
    j.append(np.array(J_inferred[c]).flatten())
    j_baseline.append(np.array(J_baseline[c]).flatten())

j_true = np.array(j_true).flatten()
j = np.array(j).flatten()
j_baseline = np.array(j_baseline).flatten()

Plot the correlation between the ground truth for the joint distribution and the infered distribution (the closer to the $x = y$ diagonal axis, the better

In [40]:
plt.figure()
plt.scatter(j_true, j, alpha=0.5)
plt.xlabel('Ground truth')
plt.ylabel('TROT (RBF)')
plt.plot(diag, diag, 'r--')

plt.show()

Plot the distribution of the error (the more packed around the origin of the $x$-axis, the better)

In [41]:
plt.figure()
bins = np.arange(-.3, .6, 0.01)
plt.hist(j_true - j, bins=bins, alpha=0.5, label='TROT')
plt.hist(j_true - j_baseline, bins=bins, alpha=0.5, label='Florida-average')
plt.legend()
plt.xlabel('Difference between inference and ground truth')

plt.show()

# Survey-based ecological inference
Same pipeline, but using a cost matrix computed thanks to the 2013 Gallup survey. (http://www.gallup.com/poll/160373/democrats-racially-diverse-republicans-mostly-white.aspx)

We assume that Gallup's Other = {Native, Other}

The cost matrix M is computed as $1-p_{ij}$, where $p_{ij}$ is the proportion of people registered to party $j$ belonging to group $i$.

In [42]:
M_sur = np.array([
               [.38, .26, .35],
               [.29, .64, .05],
               [.50, .32, .13],
               [.46, .36, .17],
               [.49, .32, .18],
               [.49, .32, .18]
               ])
M_sur = (1. - M_sur)

Once again, find the best parameters

In [43]:
best_score, best_q, best_l = CV_Local_Inference(Voters_By_County, M_sur, J, Ethnicity_Marginals, Party_Marginals,
                   CV_counties,q,l)

q: 0.50, lambda: 0.0100, KL: 0.1492, STD: 0
q: 0.50, lambda: 0.1000, KL: 0.1473, STD: 0
q: 0.50, lambda: 1.0000, KL: 0.1295, STD: 0
q: 0.50, lambda: 10.0000, KL: 0.04164, STD: 0
q: 0.50, lambda: 100.0000, KL: 0.08021, STD: 0
q: 0.60, lambda: 0.0100, KL: 0.1409, STD: 0
q: 0.60, lambda: 0.1000, KL: 0.1388, STD: 0
q: 0.60, lambda: 1.0000, KL: 0.1189, STD: 0
q: 0.60, lambda: 10.0000, KL: 0.03358, STD: 0
q: 0.60, lambda: 100.0000, KL: 0.09233, STD: 0
q: 0.70, lambda: 0.0100, KL: 0.1315, STD: 0
q: 0.70, lambda: 0.1000, KL: 0.1291, STD: 0
q: 0.70, lambda: 1.0000, KL: 0.1065, STD: 0
q: 0.70, lambda: 10.0000, KL: 0.02749, STD: 0
q: 0.70, lambda: 100.0000, KL: 0.1081, STD: 0
q: 0.80, lambda: 0.0100, KL: 0.1212, STD: 0
q: 0.80, lambda: 0.1000, KL: 0.1184, STD: 0
q: 0.80, lambda: 1.0000, KL: 0.09284, STD: 0
q: 0.80, lambda: 10.0000, KL: 0.0255, STD: 0
q: 0.80, lambda: 100.0000, KL: 0.1252, STD: 0
q: 0.90, lambda: 0.0100, KL: 0.1096, STD: 0
q: 0.90, lambda: 0.1000, KL: 0.1064, STD: 0
q: 0.90, lambd

Using these parameters, run the inference on the rest of the dataset

In [44]:
J_sur = Local_Inference(Voters_By_County, M_sur, J, Ethnicity_Marginals, Party_Marginals, all_counties, best_q, best_l)
kl, std = KL(J, J_sur, all_counties, save_to_file=False, compute_abs_err=True)

Absolute error 0.0094930050661  +  0.0


Plot correlation with ground truth

In [45]:
j_sur = []
for c in all_counties:
    j_sur.append(np.array(J_sur[c]).flatten())

j_sur = np.array(j_sur).flatten()

plt.figure()
plt.scatter(j_true, j_sur, alpha=0.5)
plt.xlabel('Ground truth')
plt.ylabel('TROT (survey)')
plt.plot(diag, diag, 'r--')

plt.show()
    

Plot error distribution (compared with Florida average)

In [46]:
plt.figure()
bins = np.arange(-.3, .6, 0.01)
plt.hist(j_true - j_sur, bins=bins, alpha=0.5, label='TROT (survey)')
plt.hist(j_true - j_baseline, bins=bins, alpha=0.5, label='Florida-average')
plt.legend()
plt.xlabel('Difference between inference and ground truth')

plt.show()